In [ ]:
# Same code as above but using Pandas
import pandas as pd
customers = spark.read.table("bronze.adventureworks.customer")
customers = customers.toPandas().drop(columns=["PasswordHash", \
"PasswordSalt", "rowguid"])
customers['ModifiedDate'] = pd. \
to_datetime(customers['ModifiedDate']).dt.strftime('%Y-%m-%d')
customers['Gender'] = customers['Title'].apply \
(lambda x: "Male" if x == "Mr." else "Female" if x == "Ms." else "Unknown")
customers['SalesPerson'] = customers['SalesPerson']. \
str.replace('adventure-works\\', '', regex=False)
customers['Phone'] = customers['Phone']. \
str.replace(r'.*?(\d{3} \d{3}-\d{4})', r'\1', regex=True)

# Check for shared phone numbers
customers['SharedPhone'] = customers.duplicated(subset=['Phone'])

# Write the cleaned data to a Delta table
customers.write.format("delta").mode("overwrite") \
.saveAsTable(schemaName + ".clean_customer")

display(customers)